<a href="https://colab.research.google.com/github/msy7822-ux/diffusers-train-controlnet/blob/main/Diffusers_train_Controlnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ControlNet学習 参考
- https://zenn.dev/mattyamonaca/articles/23cc474bc879e6
- https://huggingface.co/blog/train-your-controlnet

In [ ]:
# @title mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title hugging face login

!pip install huggingface_hub==0.22.2
# !huggingface-cli login

# NOTE: CLIよりこっちのが使いやすい
import huggingface_hub
huggingface_hub.login()

In [ ]:
# @title Install Library 1

!pip install git+https://github.com/huggingface/diffusers.git transformers accelerate xformers==0.0.17 wandb
# !wandb login

In [ ]:
# !pip install --upgrade diffusers huggingface_hub

In [ ]:
# @title clone diffusers repo

!ls
!git clone https://github.com/huggingface/diffusers.git /content/drive/MyDrive/diffusers
%cd /content/drive/MyDrive/diffusers


In [ ]:
# @title change workspace directory

%cd /content/drive/MyDrive/diffusers

In [ ]:
# @title Install Library 2

%cd /content/drive/MyDrive/diffusers/examples/controlnet

!pip install -r requirements.txt
!pip install datasets

In [ ]:
!ls /content/drive/MyDrive/diffusers/examples/datasets_v2
# !ls /content/

In [ ]:
# @title HF Script

from datasets import Dataset, load_dataset
from huggingface_hub import HfApi, HfFolder
import os
from PIL import Image

def is_image_file_by_extension(file_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    _, extension = os.path.splitext(file_path)
    return extension.lower() in image_extensions

def is_image_file(file_path):
    try:
        with Image.open(file_path) as img:
            return True
    except:
        return False

def create_dataset_from_images(path):
    """
    画像フォルダから画像のパスを読み込んでDatasetを作成する
    """

    image_folder = os.path.join(path, "target")
    conditioning_folder = os.path.join(path, "source")

    print(image_folder, conditioning_folder)

    image = [Image.open(os.path.join(image_folder, f)) for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
    conditioning = [Image.open(os.path.join(conditioning_folder, f)) for f in os.listdir(conditioning_folder) if os.path.isfile(os.path.join(conditioning_folder, f)) and (is_image_file_by_extension(os.path.join(conditioning_folder, f)))]

    caption = ["simple, single point perspective, one point perspective, anime,"]*(len(image))

    dataset = Dataset.from_dict({'image': image, 'conditioning': conditioning, 'caption': caption})
    return dataset

def upload_dataset_to_hub(dataset, dataset_name, organization=None):
    """
    DatasetをHugging Face Hubにアップロードする
    """
    # Hugging Faceの認証トークンを取得
    api = HfApi()
    token = HfFolder.get_token()
    if token is None:
        raise ValueError("Hugging Faceの認証トークンが見つかりません。huggingface-cliでログインしてください。")

    # データセットをアップロード
    if organization:
        repo_id = f"{organization}/{dataset_name}"
    else:
        repo_id = dataset_name
        print(repo_id, token)
    dataset.push_to_hub(repo_id, token=token)

# 画像が格納されているローカルフォルダのパス
image_folder = '/content/drive/MyDrive/diffusers/examples/datasets_v2'

# Datasetオブジェクトを作成
image_dataset = create_dataset_from_images(image_folder)

# DatasetをHugging Face Hubにアップロード
upload_dataset_to_hub(image_dataset, 'mf-train-cn-100-sets', 'msy78')


In [ ]:
# @title Train ControlNet

%cd /content/drive/MyDrive/diffusers/examples/controlnet

!accelerate launch train_controlnet.py \
--pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
--output_dir="model_out" \
--dataset_name=msy78/mf-train-cn-100-sets-v2 \
--conditioning_image_column=conditioning \
--image_column="image" \
--caption_column="caption" \
--resolution=512 \
--learning_rate=1e-5 \
--validation_image "./dev_controlnet/0000_211.png" "./dev_controlnet/0008_100.png" "./dev_controlnet/0010_34.png" "./dev_controlnet/0020_225.png" "./dev_controlnet/0030_91.png" "./dev_controlnet/0088_120.png" \
--validation_prompt "lineart" \
--num_train_epochs=5 \
--tracker_project_name="controlnet" \
--checkpointing_steps=1000 \
--validation_steps=1000 \
--resume_from_checkpoint 'latest' \



In [ ]:
# !pip uninstall huggingface_hub
# !pip install huggingface_hub==0.22.2

In [ ]:
!python --version
# !sudo apt install python3.12
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 1
# !sudo update-alternatives --config python3

# !apt update
# !apt upgrade
# !apt install python3.12 python3.12-dev python3.12-distutils libpython3.12-dev
# !update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 1
# !wget https://bootstrap.pypa.io/get-pip.py
# !python3.12 get-pip.py

In [ ]:
!python --version
!pip --version
print(huggingface_hub.__version__)

In [ ]:
# @title モデル学習評価

from diffusers.models.controlnet import ControlNetModel
from diffusers.pipelines.controlnet.pipeline_controlnet import StableDiffusionControlNetPipeline
from diffusers.utils.loading_utils import load_image
import torch

%cd /content/drive/MyDrive/diffusers/examples/controlnet
!ls

controlnet = ControlNetModel.from_pretrained("./model_out/checkpoint-31000/controlnet", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
).to("cuda")

# NSFW対策
pipe.safety_checker = None
pipe.requires_safety_checker = False

control_image = load_image("./test5.png")
prompt = "lineart"

generator = torch.manual_seed(3)
image = pipe(prompt, controlnet_conditioning_scale = 1.0, controlnet_conditioning_scalenum_inference_steps=20, generator=generator, image=control_image).images[0]
image.save("./output-test-5-3.png")
